# Scraping the two news sites with most coments, 20Min & Watson

In [84]:
import requests
from bs4 import BeautifulSoup
import re
import time
import twython

datestring = time.strftime("%Y-%h-%d")
datestring

'2016-Jul-09'

In [85]:
api_key = ""
api_secret = ""

access_token = ""
token_secret = ""

In [86]:
twitter = twython.Twython(api_key, api_secret, access_token, token_secret)

In [87]:
#Grab the 20Min & Watson Homepages
response = requests.get("http://www.20min.ch")
response_watson = requests.get("http://www.watson.ch/")
response_Tagi = requests.get("http://www.tagesanzeiger.ch")
response_blick = requests.get("http://www.blick.ch")

In [88]:
#Feed them into BeautifulSoup
_20Min = BeautifulSoup(response.text, 'html.parser')
Watson = BeautifulSoup(response_watson.text, 'html.parser')
Tagi = BeautifulSoup(response_Tagi.text, 'html.parser')
blick = BeautifulSoup(response_blick.text, 'html.parser')

In [89]:
#Grabbing the various Stories
stories = _20Min.find_all("div", {'id': 'content'})
stories_watson = Watson.find_all('div', {'class': 'teaser teaser_type_default     font_m  combo_bg combo_fg'})
Tagi_stories = Tagi.find_all("div", {'class': 'featureStory standard'})
Tagi_kommentare = Tagi.find_all("div", {'class': 'articleStory valued'})
blick_stories = blick.find_all('div', {'class': 'g6Slead'})

In [90]:
#Int Function
def to_int(x):
    if x == None:
        return 0
    else:
        return int(x)

In [91]:
#finding everything outside of the loop
teaser_stories = stories[0].find_all('div', {'class': "teaser "})

#starting the dictionary:
all_stories = []

#The Loop
for story in teaser_stories:
    
    #Getting the comments
    comments = story.find('div', {'class': "teaser_media front clearfix"})
    if comments:
        try:
            comment = comments.find('span', {'class': 'icon_talkback'}).next_sibling
            #Just using the number of comments
            comment = comment[:4]
            #A regular expression to make sure to get articles with more of 999 comments
            if re.search(r"\D$", comment):
                comment = int(comment[:3])
            else:
                comment = int(comment)
        except:
            comment = 1
    
    #Getting the the title
    title_div = story.find('div', {'class': "teaser_title front"})
    title_string = title_div.find('h2').string
    if re.search(r"\x96", title_string):
        title_string.replace("\x96", "")
        
    
    #Getting the HREF
    href = story.find('h3')
    href = href.find('a', href=True)
    href = href['href']
    
    #Dealing with non-full links
    if re.search(r"^http:", href):
        href = href
    else:
        href = 'http://www.20min.ch' + href
    
    #Creating the dictionary:
    dictionary = {'Site': '@20min', 'Titel': title_string, 'Kommentare': int(comment), 'Link': href, 'Date': datestring}
    
    #List of dictionaries:
    all_stories.append(dictionary)

In [92]:
#starting the dictionary:
all_stories_watson = []

#looping through titles, comments, links
for story in stories_watson:
    
    #getting HREF
    href = story.find('a', {'class': 'storylink combo_bd'})
    try:
        href = href['href']
        href = ('http://watson.ch' + href)
    except:
        href == None
        
    #Getting the story
    title = story.find('h2')
    if title == None:
        title = None
    else:
        title = title.string
    
    #Getting the Comments
    comments = story.find('a', {'class': 'comments'})
    if comments == None:
        comments = 0
    else:
        comments = story.find('img', {'src': '/media/img/main/icons/icon_comments.png'}).span.text
        comments = int(comments)
    
    #creating the dictionary
    dictionary = {'Site': '@watson_news', 'Titel': title, 'Kommentare': int(comments), 'Link': href, 'Date': datestring}
    
    #creating the list
    all_stories_watson.append(dictionary)

In [93]:
Tagi_Dict = []

#looping through all the stories
for x in Tagi_stories:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h3').get_text().strip()
    except:
        title == None

    #Getting the byline
    #time = title.find('p', {'class' : 'storyInfo'})
    
    #Getting time
    #time = x.find('p', {'class' : 'storyInfo'})
    #time = time.find('time', {'class' : 'time'})
    
    #Getting Kommentare
    if x.find('a', {'title' : 'Kommentare'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('a', {'title' : 'Kommentare'}).string
        #kommentare = kommentare.replace(u'\xa0', ' ')
        kommentare = int(re.findall(r"\d+", kommentare)[0])
        
    #Getting HREF
    try:
        href = x.find('h3')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    title = { 'Site': '@tagesanzeiger', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Dict.append(title)

In [94]:
Tagi_Kommentare_Dict = []

for x in Tagi_kommentare:
    
    #Getting each headline and extracting only the text
    try:
        title = x.find('h4').get_text().strip()
    except:
        title == None
    
    #Getting Kommentare
    if x.find('span', {'class' : 'icon icon-comment'}) is None:
        kommentare = 0
    else:
        kommentare = x.find('span', {'class' : 'icon icon-comment'}).next_sibling
        kommentare = kommentare[:4]
        #A regular expression to make sure to get articles with more of 999 comments
        if re.search(r"\D$", kommentare):
            kommentare = int(float(kommentare[:2]))
        else:
            kommentare = int(kommentare)
    #Getting HREF
    try:
        href = x.find('h4')
        href = href.find('a').get('href')
        href = 'http://www.tagesanzeiger.ch' + href
    except:
        href == None
        
    #Creating dictionary with Tagi Headlines, time
    Kommentare = { 'Site': '@tagesanzeiger', 'Titel': title, 'Kommentare': kommentare, 'Link': href, 'Date': datestring}
    
    Tagi_Kommentare_Dict.append(Kommentare)

In [95]:
all_stories_blick = []

for item in blick_stories:
    #Titel
    try:
        title = item.find('span', {'class': 'headline__item'}).string.strip()
    except:
        None
    
    #Comments
    try:
        Kommentare = item.find('span', {'class': 'mini-sharebar__comments'})
        Kommentare = Kommentare.string
    #print(Kommentare)
    except:
        None
    
    #HREF
    try:
        item.find('span', {'class': 'article__shareButton_mail'})
        href = item.find('span', {'class': 'article__shareButton_mail'})
        href = href.get('data-share-url').replace('?utm_source=email&utm_medium=social_user&utm_campaign=blick_web', "")
    except:
        None
        
    dict_items = {'Site': '@blickch', 'Titel': title, 'Kommentare': to_int(Kommentare), 'Link': href, 'Date': datestring}
    all_stories_blick.append(dict_items)
    
all_stories_blick = [dict(t) for t in set([tuple(d.items()) for d in all_stories_blick])] 

In [96]:
#adding the two dictionaries
all_stories.extend(all_stories_watson)
all_stories.extend(Tagi_Dict)
all_stories.extend(Tagi_Kommentare_Dict)
all_stories.extend(all_stories_blick)

In [97]:
#sorting the dictionary according to the value of the keys
import operator
all_stories.sort(key=operator.itemgetter('Kommentare'), reverse=True)

In [98]:
print(all_stories)

[{'Titel': 'Nationalratskommission bittet Ältere zur Kasse', 'Link': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162', 'Kommentare': 780, 'Date': '2016-Jul-09', 'Site': '@20min'}, {'Titel': 'Deutsche Chefs wollen nicht mehr in die Schweiz', 'Link': 'http://www.20min.ch/finance/news/story/Deutsche-Chefs-wollen-nicht-mehr-in-die-Schweiz-24079080', 'Kommentare': 642, 'Date': '2016-Jul-09', 'Site': '@20min'}, {'Titel': 'Lohnt es sich, den Motor am Rotlicht abzuschalten?', 'Link': 'http://www.20min.ch/schweiz/energy-challenge/story/Lohnt-es-sich--den-Motor-am-Rotlicht-abzuschalten--21177214', 'Kommentare': 595, 'Date': '2016-Jul-09', 'Site': '@20min'}, {'Titel': 'Krankenkassenprämien steigen immer mehr', 'Link': 'http://www.20min.ch/finance/news/story/Krankenkassenpraemien-steigen-immer-mehr-24655934', 'Kommentare': 396, 'Date': '2016-Jul-09', 'Site': '@20min'}, {'Titel': '99+ Bilder und Gifs gegen Langeweile: Der Picdump ist da!', 'Link': 'ht

In [99]:
len(all_stories )
for x in all_stories:
    print(x['Site'], x['Titel'])

@20min Nationalratskommission bittet Ältere zur Kasse
@20min Deutsche Chefs wollen nicht mehr in die Schweiz
@20min Lohnt es sich, den Motor am Rotlicht abzuschalten?
@20min Krankenkassenprämien steigen immer mehr
@watson_news 99+ Bilder und Gifs gegen Langeweile: Der Picdump ist da!
@20min «So sehen Bünzli aus, die am Rasenmähen sind»
@20min Salt-Abos fliegen bei Mobilezone raus
@20min Was kann man tun, wenn ein Flug annulliert wird?
@20min «Mein Freund kommt extra für mich in die Schweiz»
@20min Darum hatte Salt gestern einen Totalausfall
@20min Bomben-Roboter ist die neue Waffe der Polizei
@20min Tropennacht leitet heisses Wochenende ein
@20min UPC-Kunden müssen ihr Passwort ändern
@20min Finden Sie heraus, wie Sie mit dem Chef reden sollten
@20min «Die Zustände in den Camps gehen mir nahe»
@watson_news Weil ihr so schön brav wart: 101 Bilder für ein besseres Wochenende – EXTRA PICDUMP
@20min «Statt AKWs braucht es Tausende Batterien»
@20min «Brauche jedes Jahr 31'000 Euro für Handy

In [100]:
all_stories[0]

{'Date': '2016-Jul-09',
 'Kommentare': 780,
 'Link': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162',
 'Site': '@20min',
 'Titel': 'Nationalratskommission bittet Ältere zur Kasse'}

In [101]:
all_stories[0]['Titel']

'Nationalratskommission bittet Ältere zur Kasse'

In [102]:
len(all_stories)

247

In [103]:
Sum = 0
for item in all_stories:
    Sum = Sum + item['Kommentare']

In [104]:
Sum

6621

In [105]:
Anteil = round(all_stories[0]['Kommentare']/Sum * 100)
Anteil

12

In [106]:
Kommentar_Schnitt = round(Sum / len(all_stories))

In [107]:
Count = all_stories[0]['Kommentare']
#if Count > 1100:
#    Comment_Count = "Lesen. Dazu hat Krethi und Plethi eine Meinung."
#elif Count > 900:
#    Comment_Count = "Party-Talk. Wirklich an jeder Party."
#elif Count > 700:
#    Comment_Count = "Da haben am Stammtisch einige etwas dazu zu sagen."
#elif Count > 500:
#    Comment_Count = "Das bewegt einge. Am richtig viele sind das noch nicht."
#elif Count > 0:
#    Comment_Count = "Da schlafen einige fleissige Kommentarschreiben wohl noch."

In [108]:
#sentence = 'Kommentare: {}. Quelle: {}. {} {}'.format(all_stories[0]['Kommentare'], all_stories[0]['Site'], Comment_Count, all_stories[0]['Link'])

In [109]:
#sentence = 'Die heisseste Front-Story läuft {} mit {} Kommentaren. Das Total liegt derzeit bei 6608 Reaktionen, {} pro Story. {}'.format(all_stories[0]['Site'], all_stories[0]['Kommentare'], Kommentar_Schnitt, all_stories[0]['Link']) 

In [110]:
sentence = 'Die heisseste Front-Story läuft {} mit {} Kommentaren. Das sind {}% von total {} Reaktionen. {}'.format(all_stories[0]['Site'], all_stories[0]['Kommentare'], Anteil, Sum, all_stories[0]['Link'])

In [111]:
print(sentence)

Die heisseste Front-Story läuft @20min mit 780 Kommentaren, 12% von aktuell total 6621 Reaktionen. http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162


In [112]:
#twitter = twython.Twython(api_key, api_secret)
#auth = twitter.get_authentication_tokens()
#print("Log into Twitter as the user you want to authorize and visit this URL:")
#print("\t" + auth['auth_url'])

In [113]:
#pin = "XXXXXX"
#
#twitter = twython.Twython(api_key, api_secret, auth['oauth_token'], auth['oauth_token_secret'])
#tokens = twitter.get_authorized_tokens(pin)
#
#new_access_token = tokens['oauth_token']
#new_token_secret = tokens['oauth_token_secret']
#print("your access token:", new_access_token)
#print("your token secret:", new_token_secret)

In [114]:
twitter.update_status(status=sentence)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Jul 10 01:19:31 +0000 2016',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': '20min.ch/schweiz/news/s…',
    'expanded_url': 'http://www.20min.ch/schweiz/news/story/Nationalratskommission-bittet--ltere-zur-Kasse-16499162',
    'indices': [99, 122],
    'url': 'https://t.co/xkafvQCw7W'}],
  'user_mentions': [{'id': 15786941,
    'id_str': '15786941',
    'indices': [32, 38],
    'name': '20 Minuten',
    'screen_name': '20min'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 751948936789778432,
 'id_str': '751948936789778432',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'de',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://www.barnabyskinner.com/" rel="nofo